In [259]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
import pickle


In [260]:
#Grunddaten
Wetterdaten = pd.read_csv("../02_Cleaned/Features/01Wetterdaten_cleaned.csv")
Verkehrsdaten = pd.read_csv("../02_Cleaned/Features/02Verkehrsdaten.csv")
Parkhäuser = pd.read_csv("../02_Cleaned/Features/03Parkhäuser.csv")
Flugbewegungen = pd.read_csv("../02_Cleaned/Features/04Airport_traffic_hourly_cleaned.csv")
Gasverbrauch = pd.read_csv("../02_Cleaned/Features/05Gasverbrauch_cleaned.csv")
Stromverbrauch = pd.read_csv("../02_Cleaned/Features/06Stromverbrauch_hourly_cleaned.csv")
#Zusätzliche Daten (Luftdaten von anderen Stationen in Basel)                                        Müssen die als lagged values mit einfliessen, weil man sonst die Daten der Zükunft hätte (vielleicht auch sonst viel zu hohe Korrelation)
Basel_Binningen = pd.read_csv("../02_Cleaned/Target/Luft_Basel-Binningen_cleaned.csv")
Chrischona = pd.read_csv("../02_Cleaned/Target/Luft_Chrischona_cleaned.csv")
Feldbergstrasse = pd.read_csv("../02_Cleaned/Target/Luft_Feldbergstrasse_cleaned.csv")


In [261]:
# Ensure the index is a DatetimeIndex (it should already be with UTC timezone)
Basel_Binningen.index = pd.to_datetime(Basel_Binningen.index, utc=True)
Chrischona.index = pd.to_datetime(Chrischona.index, utc=True)
Feldbergstrasse.index = pd.to_datetime(Feldbergstrasse.index, utc=True)

# Create a one-hour lag for the specified dataframes
Basel_Binningen_lagged = Basel_Binningen.shift(periods=1, freq='h')
Chrischona_lagged = Chrischona.shift(periods=1, freq='h')
Feldbergstrasse_lagged = Feldbergstrasse.shift(periods=1, freq='h')

Basel_Binningen.reset_index(inplace=True)
Chrischona.reset_index(inplace=True)
Feldbergstrasse.reset_index(inplace=True)

# Display the first few rows of the lagged dataframes
print(Basel_Binningen_lagged.head())
print(Chrischona_lagged.head())
print(Feldbergstrasse_lagged.head())


                                                         Datum  O3 [ug/m3]  \
1970-01-01 01:00:00+00:00            2022-01-01 00:00:00+00:00         6.2   
1970-01-01 01:00:00.000000001+00:00  2022-01-01 01:00:00+00:00         3.7   
1970-01-01 01:00:00.000000002+00:00  2022-01-01 02:00:00+00:00         3.7   
1970-01-01 01:00:00.000000003+00:00  2022-01-01 03:00:00+00:00         4.1   
1970-01-01 01:00:00.000000004+00:00  2022-01-01 04:00:00+00:00         2.4   

                                     NO2 [ug/m3]  PM10 [ug/m3]  PM2.5 [ug/m3]  \
1970-01-01 01:00:00+00:00                   16.9          34.3           34.1   
1970-01-01 01:00:00.000000001+00:00         18.4          59.6           56.8   
1970-01-01 01:00:00.000000002+00:00         16.8          84.9           82.6   
1970-01-01 01:00:00.000000003+00:00         16.5          60.0           58.8   
1970-01-01 01:00:00.000000004+00:00         20.8          46.2           44.2   

                                     CPC [1/

In [262]:

Parkhäuser.rename(columns={"RoundedTime": "Datum"}, inplace=True)
Parkhäuser.head()
#Flugbewegungen.drop(columns=["Unnamed: 0"], inplace=True)
Flugbewegungen.head()
#Gasverbrauch.head()
#Schallmessungen.drop(columns=["Unnamed: 0"], inplace=True)
#Schallmessungen.head()
#Stromverbrauch.drop(columns=["Unnamed: 0"], inplace=True)
#Stromverbrauch.head()
#Wetterdaten.head()
#Verkehrsdaten.rename(columns={"DateTimeFrom": "Datum"}, inplace=True)
#Verkehrsdaten.head()

Parkhäuser.head()
Verkehrsdaten = Verkehrsdaten.rename(columns={"DateTimeFrom": "Datum"})
print(Verkehrsdaten['Datum'].isna().sum())




0


In [263]:
#standardise the data


dfs = [Parkhäuser, Flugbewegungen, Gasverbrauch, Stromverbrauch, Wetterdaten, Verkehrsdaten, Basel_Binningen, Chrischona, Feldbergstrasse]

# Dictionary to store the mean and standard deviation for each dataframe
scalers = {}

# Loop through each dataframe
for i, df in enumerate(dfs):
    numeric_cols = df.select_dtypes(include=[np.number])
    if not numeric_cols.empty:
        if i == 1:  # Exclude 'Hour' column for Flugbewegungen
            numeric_cols = numeric_cols.drop(columns=['Hour'])
        print(f"Processing df{i} with columns: {numeric_cols.columns}")
        scaler = StandardScaler()
        # Fit the scaler on the dataframe and transform the data
        scaled_data = scaler.fit_transform(numeric_cols)
        
        # Save the mean and standard deviation
        scalers[f'df{i}'] = {'mean': scaler.mean_, 'scale': scaler.scale_}
        
        # Replace the original data with the standardized data
        df[numeric_cols.columns] = scaled_data
        dfs[i] = df
    else:
        print(f"df{i} has no numeric columns or is empty.")

# Assign the transformed dataframes back to their original names
Parkhäuser, Flugbewegungen, Gasverbrauch, Stromverbrauch, Wetterdaten, Verkehrsdaten, Basel_Binningen, Chrischona, Feldbergstrasse = dfs

# Save the scalers dictionary for later use
with open('../04_WorkingDatasets/NormalData/scalers.pkl', 'wb') as f:
    pickle.dump(scalers, f)


Processing df0 with columns: Index(['Rebgassechange', 'Clarahuuschange', 'Citychange', 'Storchenchange',
       'Post Baselchange', 'Aeschenchange', 'Bahnhof Südchange',
       'Bad. Bahnhofchange', 'Europechange', 'Claramattechange',
       'Elisabethenchange', 'Steinenchange', 'Kunstmuseumchange',
       'Messechange', 'Anfoschange', 'Centralbahnparkingchange'],
      dtype='object')
Processing df1 with columns: Index(['Unnamed: 0', 'Traffic'], dtype='object')
Processing df2 with columns: Index(['Gasverbrauch'], dtype='object')
Processing df3 with columns: Index(['Stromverbrauch'], dtype='object')
Processing df4 with columns: Index(['Unnamed: 0', 'Basel Temperature [2 m elevation corrected]',
       'Basel Precipitation Total', 'Basel Wind Speed [10 m]',
       'Basel Wind Direction [10 m]'],
      dtype='object')
Processing df5 with columns: Index(['350n_sumPW', '350n_sumLief', '350n_sumLW', '350v_sumPW',
       '350v_sumLief', '350v_sumLW', '352n_sumPW', '352n_sumLief',
       '352

In [264]:
print(type(Verkehrsdaten))
Wetterdaten.drop(columns=["Unnamed: 0"], inplace=True)
Flugbewegungen.drop(columns=["Unnamed: 0"], inplace=True)
Gasverbrauch.sort_values(by=["Datum"], inplace=True)

<class 'pandas.core.frame.DataFrame'>


In [265]:

Flugbewegungen["Datum"].dtype

dtype('O')

In [266]:
# Merge all dataframes on the 'Datum' column
final_df = Parkhäuser.merge(Wetterdaten, on='Datum', how='inner') \
                      .merge(Verkehrsdaten, on='Datum', how='inner') \
                      .merge(Gasverbrauch, on='Datum', how='inner') \
                      .merge(Flugbewegungen, on='Datum', how='inner') \
                      .merge(Stromverbrauch, on='Datum', how='inner') \
                      .merge(Chrischona_lagged, on='Datum', how='inner') \
                      .merge(Feldbergstrasse_lagged, on='Datum', how='inner') \
                      .merge(Basel_Binningen_lagged, on='Datum', how='inner') 


final_df.head(20)

,Datum,Rebgassechange,Clarahuuschange,Citychange,Storchenchange,Post Baselchange,Aeschenchange,Bahnhof Südchange,Bad. Bahnhofchange,Europechange,Claramattechange,Elisabethenchange,Steinenchange,Kunstmuseumchange,Messechange,Anfoschange,Centralbahnparkingchange,Basel Temperature [2 m elevation corrected],Basel Precipitation Total,Basel Wind Speed [10 m],Basel Wind Direction [10 m],350n_sumPW,350n_sumLief,350n_sumLW,350v_sumPW,350v_sumLief,350v_sumLW,352n_sumPW,352n_sumLief,352n_sumLW,352v_sumPW,352v_sumLief,352v_sumLW,354n_sumPW,354n_sumLief,354n_sumLW,354v_sumPW,354v_sumLief,354v_sumLW,402v_sumPW,402v_sumLief,402v_sumLW,402n_sumPW,402n_sumLief,402n_sumLW,403n_sumPW,403n_sumLief,403n_sumLW,403v_sumPW,403v_sumLief,403v_sumLW,405n_sumPW,405n_sumLief,405n_sumLW,405v_sumPW,405v_sumLief,405v_sumLW,406n_sumPW,406n_sumLief,406n_sumLW,406v_sumPW,406v_sumLief,406v_sumLW,407n_sumPW,407n_sumLief,407n_sumLW,408n_sumPW,408n_sumLief,408n_sumLW,409n_sumPW,409n_sumLief,409n_sumLW,409v_sumPW,409v_sumLief,409v_sumLW,410v_sumPW,410v_sumLief,410v_sumLW,410n_sumPW,410n_sumLief,410n_sumLW,411n_sumPW,411n_sumLief,411n_sumLW,411v_sumPW,411v_sumLief,411v_sumLW,412v_sumPW,412v_sumLief,412v_sumLW,413n_sumPW,413n_sumLief,413n_sumLW,415v_sumPW,415v_sumLief,415v_sumLW,415n_sumPW,415n_sumLief,415n_sumLW,417n_sumPW,417n_sumLief,417n_sumLW,419v_sumPW,419v_sumLief,419v_sumLW,419n_sumPW,419n_sumLief,419n_sumLW,420n_sumPW,420n_sumLief,420n_sumLW,420v_sumPW,420v_sumLief,420v_sumLW,653v_sumPW,653v_sumLief,653v_sumLW,653n_sumPW,653n_sumLief,653n_sumLW,656v_sumPW,656v_sumLief,656v_sumLW,659v_sumPW,659v_sumLief,659v_sumLW,659n_sumPW,659n_sumLief,659n_sumLW,660v_sumPW,660v_sumLief,660v_sumLW,660n_sumPW,660n_sumLief,660n_sumLW,84111104n_sumPW,84111104n_sumLief,84111104n_sumLW,84111104v_sumPW,84111104v_sumLief,84111104v_sumLW,84111108n_sumPW,84111108n_sumLief,84111108n_sumLW,84111108v_sumPW,84111108v_sumLief,84111108v_sumLW,Gasverbrauch,Hour,Traffic,Stromverbrauch,o3_stundenmittelwerte_ug_m3,pm10_stundenmittelwerte_ug_m3,pm2_5_stundenmittelwerte_ug_m3,no2_stundenmittelwerte_ug_m3,O3 [ug/m3],NO2 [ug/m3],PM10 [ug/m3],PM2.5 [ug/m3],CPC [1/cm3],PREC [mm],RAD [W/m2],SO2 [ug/m3],NOX [ug/m3 eq. NO2],TEMP [C]
0,2022-01-01 00:00:00+00:00,-0.489808,-1.162325,-0.163786,-1.110112,-0.000054,-0.258093,-0.245491,-0.158272,-0.151270,-1.607696,-0.077419,-0.232976,-0.177704,-0.175812,-0.000238,-0.115383,-0.926467,-0.314805,-0.694066,0.115891,-0.546185,-0.969412,-0.629404,-0.728106,-0.772489,-0.731284,-0.141373,-0.645283,-1.019502,-0.253232,-0.810792,-1.124504,-0.044382,-0.566314,-0.779749,0.210334,-0.758520,-0.294615,-0.903982,-1.083289,-1.044061,-0.703070,-0.807164,-1.103759,-0.181033,-0.740095,-0.928990,-0.456383,-0.845842,-0.851382,-0.471361,-0.836661,-0.766543,-0.304907,-0.787969,-0.634854,-0.566552,-0.851172,-0.898361,-0.366723,-0.920238,-0.728092,-0.507486,-0.865240,-0.712555,-0.266243,-0.828557,-1.390464,-0.733559,-0.822739,-1.205729,-0.823698,-0.964702,-1.128021,-0.543488,-0.574552,-0.962667,-0.654790,-0.792814,-0.843871,-0.619298,-0.716948,-1.266743,-0.713230,-0.797324,-1.236077,-0.594955,-0.693668,-0.977663,-0.500056,-0.822865,-0.844158,-0.556320,-0.883664,-1.044548,-0.513266,-0.908405,-0.928473,-0.266213,-0.432272,-1.066476,-0.116773,-0.818457,-0.848324,-0.086645,-0.703054,-0.805670,-0.438091,-0.912092,-0.856912,-0.591225,-0.828177,-1.003686,-0.948583,-0.849098,-1.050331,-0.768264,-0.851954,-1.149120,-1.003866,-0.791878,-0.691576,-0.901352,-0.832350,-0.866946,-0.973694,-0.921841,-0.818372,-1.540182,-1.612023,-1.300587,-1.679601,-1.554684,-1.288546,-0.696574,2.164977,-0.082335,-1.362775,1.404508,-0.117866,-0.685184,-0.566797,-1.055921,-0.813705,-0.413276,-1.072953,0.294494,0,-1.142151,-1.291844,54.712,85.530,58.362,22.670,6.2,16.9,34.3,34.1,30241.1,0.0,0.0,1.2,26.0,5.0
1,2022-01-01 01:00:00+00:00,-0.489808,-0.193898,-0.043944,-0.525891,-0.191296,-0.000173,-0.000190,-0.000462,-0.000203,-0.000156,-0.139251,-0.442487,-0.000274,-0.200896,-0.056599,-0.057814,-1.041967,-0.314805,-0.91

In [267]:
print(final_df.tail(25))

                           Datum  Rebgassechange  Clarahuuschange  Citychange  \
24549  2024-10-19 21:00:00+00:00       -0.979741        -0.000213   -0.632262   
24550  2024-10-19 22:00:00+00:00       -1.399684        -0.000213   -0.250945   
24551  2024-10-19 23:00:00+00:00       -1.119722        -0.968639   -0.109312   
24552  2024-10-20 00:00:00+00:00       -0.139856        -0.000213   -0.076628   
24553  2024-10-20 01:00:00+00:00        0.000125        -0.000213   -0.022154   
24554  2024-10-20 02:00:00+00:00        0.000125        -0.000213    0.010530   
24555  2024-10-20 03:00:00+00:00        0.000125        -0.000213   -0.000365   
24556  2024-10-20 04:00:00+00:00        0.070116         0.968214    0.860323   
24557  2024-10-20 05:00:00+00:00        0.000125        -0.000213   -0.022154   
24558  2024-10-20 06:00:00+00:00        0.140106        -0.000213    0.032320   
24559  2024-10-20 07:00:00+00:00        0.560049        -0.000213    0.250215   
24560  2024-10-20 08:00:00+0

In [268]:
test_dataFrame = [Parkhäuser,
                Wetterdaten,
                Flugbewegungen,
                Gasverbrauch,
                Stromverbrauch,
                Verkehrsdaten]

for i in test_dataFrame:
    print(i['Datum'].max())

2024-10-21 14:00:00+00:00
2024-11-22 23:00:00+00:00
2024-11-22 23:00:00+00:00
2024-11-24 04:00:00+00:00
2024-11-20 22:00:00+00:00
2024-10-20 21:00:00+00:00


In [269]:
final_df.to_csv("../04_WorkingDatasets/NormalData/Working_DataFrame.csv", index=False)

In [270]:

Stromverbrauch["Datum"].dtype
Wetterdaten["Datum"].dtype
Gasverbrauch["Datum"].dtype

Flugbewegungen["Datum"] = Flugbewegungen["Datum"].astype(str)
Flugbewegungen["Datum"].dtype

dtype('O')

In [271]:
Gasverbrauch.sort_values(by=["Datum"], inplace=True)
Gasverbrauch.tail()
Stromverbrauch.sort_values(by=["Datum"], inplace=True)
Stromverbrauch.tail()
Wetterdaten.sort_values(by=["Datum"], inplace=True)
Wetterdaten.tail()
Flugbewegungen.sort_values(by=["Datum"], inplace=True) 
Flugbewegungen.tail()

,Datum,Hour,Traffic
25363,2024-11-22 19:00:00+00:00,19,0.084474
25364,2024-11-22 20:00:00+00:00,20,0.391130
25365,2024-11-22 21:00:00+00:00,21,0.544459
25366,2024-11-22 22:00:00+00:00,22,0.084474
25367,2024-11-22 23:00:00+00:00,23,-0.528839


**Check if there are missing timestamps in the dataset**


In [272]:

df = final_df.copy()
# Ensure 'Datum' is a datetime object
df['Datum'] = pd.to_datetime(df['Datum'])

# Create a complete range of hourly timestamps from the first to the last point
full_range = pd.date_range(start=df['Datum'].min(), end=df['Datum'].max(), freq='h')

# Identify missing timestamps
missing_timestamps = full_range.difference(df['Datum'])

# Display the results
if missing_timestamps.empty:
    print("No missing hourly timestamps. All hours are accounted for.")
else:
    print("Missing hourly timestamps:")
    print(missing_timestamps)

No missing hourly timestamps. All hours are accounted for.


In [273]:
# Check for 0 values, NaN values, and '/N' values in all columns
zero_values = (df == 0).sum()
nan_values = df.isna().sum()
slash_n_values = (df == '/N').sum()

# Print the results
print("Zero values in each column:")
print(zero_values)
print("\nNaN values in each column:")
print(nan_values)
print("\n'/N' values in each column:")
print(slash_n_values)

Zero values in each column:
Datum                                              0
Rebgassechange                                     0
Clarahuuschange                                    0
Citychange                                         0
Storchenchange                                     0
Post Baselchange                                   0
Aeschenchange                                      0
Bahnhof Südchange                                  0
Bad. Bahnhofchange                                 0
Europechange                                       0
Claramattechange                                   0
Elisabethenchange                                  0
Steinenchange                                      0
Kunstmuseumchange                                  0
Messechange                                        0
Anfoschange                                        0
Centralbahnparkingchange                           0
Basel Temperature [2 m elevation corrected]        0
Basel Precipitatio

# PART 2 Feature Combination

In [274]:
features = final_df
target = pd.read_csv("../02_Cleaned/Target/11Target Additive_Combined_Resid_Trend_Target.csv")

In [275]:
target.to_csv("../04_WorkingDatasets/NormalData/Target_Additive.csv", index=False)

In [276]:
# Define all features based on the columns of the features DataFrame
all_features = features.columns.tolist()

# Create a list to store the new features and their correlations
new_feature_correlations = []

# Create a set to keep track of added features
added_features = set()

# Dictionary to store new features
new_features_dict = {}

# Iterate over each pair of features to create new features
for i in range(len(all_features)):
    for j in range(i, len(all_features)):
        feature1 = all_features[i]
        feature2 = all_features[j]
        
        # Skip the 'Datum' column
        if feature1 == 'Datum' or feature2 == 'Datum':
            continue
        
        # Create new features by multiplying and calculating growth
        new_feature_name = f'{feature1}_x_{feature2}'
        new_feature_growth_name = f'{feature1}_growth'
        
        # Multiply features
        new_features_dict[new_feature_name] = features[feature1] * features[feature2]
        
        # Calculate growth (difference to last period)
        new_features_dict[new_feature_growth_name] = features[feature1].diff()

        # Calculate growth (difference to last period) of the multiplied features
        new_feature_mult_growth_name = f'{new_feature_name}_growth'
        new_features_dict[new_feature_mult_growth_name] = new_features_dict[new_feature_name].diff()
        
        # Calculate correlations with the target variable
        correlation_mult = new_features_dict[new_feature_name].corr(target['PM10_Combined_Trend_Residual'])
        correlation_growth = new_features_dict[new_feature_growth_name].corr(target['PM10_Combined_Trend_Residual'])
         
        # Store the correlations if they are high and not already added
        if abs(correlation_mult) > 0.1 and new_feature_name not in added_features:
            new_feature_correlations.append((new_feature_name, correlation_mult))
            added_features.add(new_feature_name)
        if abs(correlation_growth) > 0.1 and new_feature_growth_name not in added_features:
            new_feature_correlations.append((new_feature_growth_name, correlation_growth))
            added_features.add(new_feature_growth_name)

# Concatenate new features to the original DataFrame
features = pd.concat([features, pd.DataFrame(new_features_dict)], axis=1)

# Print the new features with high correlations
for feature, corr in new_feature_correlations:
    print(f'Feature: {feature}, Correlation: {corr}')


Feature: Basel Temperature [2 m elevation corrected]_x_Basel Temperature [2 m elevation corrected], Correlation: 0.2958803126640731
Feature: Basel Temperature [2 m elevation corrected]_x_Basel Wind Speed [10 m], Correlation: 0.16492811352292927
Feature: Basel Temperature [2 m elevation corrected]_x_660n_sumPW, Correlation: 0.11947271525695306
Feature: Basel Temperature [2 m elevation corrected]_x_Gasverbrauch, Correlation: -0.33163000210476223
Feature: Basel Temperature [2 m elevation corrected]_x_Traffic, Correlation: 0.1068350172287397
Feature: Basel Temperature [2 m elevation corrected]_x_pm10_stundenmittelwerte_ug_m3, Correlation: -0.19046684058629706
Feature: Basel Temperature [2 m elevation corrected]_x_pm2_5_stundenmittelwerte_ug_m3, Correlation: -0.22713850479289824
Feature: Basel Temperature [2 m elevation corrected]_x_no2_stundenmittelwerte_ug_m3, Correlation: -0.11112827695080678
Feature: Basel Temperature [2 m elevation corrected]_x_NO2 [ug/m3], Correlation: -0.286847867985

**Part 3** Selecting the 50 most important features for Model stability

In [277]:
#reimport the data to solve dependencies
working_dataframe = pd.read_csv('../04_WorkingDatasets/NormalData/Working_DataFrame.csv')
target_dataframe = pd.read_csv('../04_WorkingDatasets/NormalData/Target_Additive.csv')

In [278]:
from sklearn.feature_selection import SelectKBest, f_regression

# Select only numeric columns from working_dataframe
working_dataframe_numeric = working_dataframe.select_dtypes(include=[float, int])

# Add 'Datum' column back if it exists in the original dataframe
if 'Datum' in working_dataframe.columns:
    working_dataframe_numeric['Datum'] = working_dataframe['Datum']

# Define the feature columns and target column
feature_columns = working_dataframe_numeric.columns.tolist()
if 'Datum' in feature_columns:
    feature_columns.remove('Datum')
target_column = 'PM10_Combined_Trend_Residual'

# Align the indices of working_dataframe_numeric and target_dataframe
aligned_data = working_dataframe_numeric.join(target_dataframe[target_column], how='inner')

# Prepare the data
X = aligned_data[feature_columns]
y = aligned_data[target_column]

# Initialize SelectKBest with the scoring function and the desired number of features
selector = SelectKBest(score_func=f_regression, k=50)

# Fit the selector
selector.fit(X, y)

# Get the selected features
selected_features = [feature for feature, support in zip(feature_columns, selector.get_support()) if support]

# Create a new dataframe with the selected features
if 'Datum' in working_dataframe.columns:
    reduced_working_dataframe = working_dataframe_numeric[selected_features + ['Datum']]
else:
    reduced_working_dataframe = working_dataframe_numeric[selected_features]

print("Selected features:", selected_features)
print("Reduced dataframe shape:", reduced_working_dataframe.shape)
print("Selected features as list:", selected_features)
for feature in selected_features:
    print(feature)


Selected features: ['Basel Temperature [2 m elevation corrected]', 'Basel Precipitation Total', 'Basel Wind Speed [10 m]', 'Basel Wind Direction [10 m]', '350v_sumLW', '352v_sumPW', '352v_sumLief', '402v_sumPW', '402n_sumLief', '402n_sumLW', '403v_sumPW', '403v_sumLW', '405v_sumLief', '405v_sumLW', '406n_sumLW', '406v_sumLief', '406v_sumLW', '408n_sumPW', '408n_sumLief', '408n_sumLW', '415v_sumLW', '417n_sumPW', '417n_sumLief', '417n_sumLW', '419v_sumLW', '419n_sumPW', '419n_sumLief', '420n_sumLW', '653n_sumLW', '659v_sumLW', '659n_sumLW', '660v_sumPW', '660n_sumPW', '84111104n_sumLief', '84111104v_sumLief', 'Gasverbrauch', 'Traffic', 'Stromverbrauch', 'pm10_stundenmittelwerte_ug_m3', 'pm2_5_stundenmittelwerte_ug_m3', 'no2_stundenmittelwerte_ug_m3', 'O3 [ug/m3]', 'NO2 [ug/m3]', 'PM10 [ug/m3]', 'PM2.5 [ug/m3]', 'CPC [1/cm3]', 'PREC [mm]', 'SO2 [ug/m3]', 'NOX [ug/m3 eq. NO2]', 'TEMP [C]']
Reduced dataframe shape: (24574, 51)
Selected features as list: ['Basel Temperature [2 m elevation c

In [279]:
# Reorder columns to make 'Datum' the first column
columns = ['Datum'] + [col for col in reduced_working_dataframe.columns if col != 'Datum']
reduced_working_dataframe = reduced_working_dataframe[columns]

# Export the reduced working dataframe with the most relevant features to a CSV file
reduced_working_dataframe.to_csv('../04_WorkingDatasets/Top50Data/50MostImpFeatures_DF.csv', index=False)




# Align the target dataframe with the reduced working dataframe
aligned_target_dataframe = target.loc[reduced_working_dataframe.index]

# Export the aligned target dataframe to a CSV file
aligned_target_dataframe.to_csv('../04_WorkingDatasets/Top50Data/TargetCutto50MostImpFeatures_DF.csv', index=False)


# Getting also the top ten features

In [280]:
from sklearn.feature_selection import SelectKBest, f_regression

# Select only numeric columns from working_dataframe
working_dataframe_numeric = working_dataframe.select_dtypes(include=[float, int])

# Add 'Datum' column back if it exists in the original dataframe
if 'Datum' in working_dataframe.columns:
    working_dataframe_numeric['Datum'] = working_dataframe['Datum']

# Define the feature columns and target column
feature_columns = working_dataframe_numeric.columns.tolist()
if 'Datum' in feature_columns:
    feature_columns.remove('Datum')
target_column = 'PM10_Combined_Trend_Residual'

# Align the indices of working_dataframe_numeric and target_dataframe
aligned_data = working_dataframe_numeric.join(target_dataframe[target_column], how='inner')

# Prepare the data
X = aligned_data[feature_columns]
y = aligned_data[target_column]

# Initialize SelectKBest with the scoring function and the desired number of features
selector = SelectKBest(score_func=f_regression, k=50)

# Fit the selector
selector.fit(X, y)

# Get the scores and selected features
scores = selector.scores_
selected_features = [feature for feature, support in zip(feature_columns, selector.get_support()) if support]
selected_scores = [score for score, support in zip(scores, selector.get_support()) if support]

# Sort the selected features by their scores in descending order
sorted_features_scores = sorted(zip(selected_features, selected_scores), key=lambda x: x[1], reverse=True)
sorted_features = [feature for feature, score in sorted_features_scores]

# Create a new dataframe with the selected features
if 'Datum' in working_dataframe.columns:
    reduced_working_dataframe = working_dataframe_numeric[sorted_features + ['Datum']]
else:
    reduced_working_dataframe = working_dataframe_numeric[sorted_features]

print("Selected features (ordered by score):", sorted_features)
print("Reduced dataframe shape:", reduced_working_dataframe.shape)
print("Selected features as list (ordered by score):", sorted_features)
for feature in sorted_features:
    print(feature)

    # Add the score to each feature
    features_with_scores = {feature: score for feature, score in sorted_features_scores}

    print("Selected features with scores (ordered by score):")
    for feature, score in features_with_scores.items():
        print(f"{feature}: {score}")
# Reorder columns to make 'Datum' the first column
columns = ['Datum'] + [col for col in reduced_working_dataframe.columns if col != 'Datum']
reduced_working_dataframe = reduced_working_dataframe[columns]

# Export the reduced working dataframe with the most relevant features to a CSV file
#reduced_working_dataframe.to_csv('50MostImpFeatures_DF.csv', index=False)              -> war oben schonmal ????????????????????
# Align the target dataframe with the reduced working dataframe
aligned_target_dataframe = target_dataframe.loc[reduced_working_dataframe.index]

# Export the aligned target dataframe to a CSV file
#aligned_target_dataframe.to_csv('TargetCutto50MostImpFeatures_DF.csv', index=False)    -> war oben schonmal ????????????????????
top_10 = working_dataframe.copy()
top_10_features = sorted_features[:10]
print("Top 10 features:", top_10_features)
top_10_good = top_10.iloc[:, top_10.columns.isin(top_10_features)].copy()
top_10_good.head()

Selected features (ordered by score): ['pm10_stundenmittelwerte_ug_m3', 'pm2_5_stundenmittelwerte_ug_m3', 'PM10 [ug/m3]', 'PM2.5 [ug/m3]', 'NO2 [ug/m3]', 'NOX [ug/m3 eq. NO2]', 'no2_stundenmittelwerte_ug_m3', 'CPC [1/cm3]', 'Basel Wind Direction [10 m]', 'Basel Wind Speed [10 m]', 'Gasverbrauch', 'Basel Precipitation Total', 'SO2 [ug/m3]', 'O3 [ug/m3]', 'TEMP [C]', 'Basel Temperature [2 m elevation corrected]', '408n_sumPW', 'PREC [mm]', '84111104n_sumLief', '408n_sumLief', '84111104v_sumLief', 'Stromverbrauch', '417n_sumPW', '408n_sumLW', '402v_sumPW', '403v_sumLW', '405v_sumLW', '402n_sumLW', '403v_sumPW', '406v_sumLW', '660n_sumPW', '660v_sumPW', '350v_sumLW', '417n_sumLW', '352v_sumPW', '415v_sumLW', '417n_sumLief', '419n_sumLief', '352v_sumLief', '420n_sumLW', '402n_sumLief', 'Traffic', '659n_sumLW', '406v_sumLief', '653n_sumLW', '659v_sumLW', '419n_sumPW', '406n_sumLW', '405v_sumLief', '419v_sumLW']
Reduced dataframe shape: (24574, 51)
Selected features as list (ordered by score)

,Basel Wind Speed [10 m],Basel Wind Direction [10 m],pm10_stundenmittelwerte_ug_m3,pm2_5_stundenmittelwerte_ug_m3,no2_stundenmittelwerte_ug_m3,NO2 [ug/m3],PM10 [ug/m3],PM2.5 [ug/m3],CPC [1/cm3],NOX [ug/m3 eq. NO2]
0,-0.694066,0.115891,85.530,58.362,22.670,16.9,34.3,34.1,30241.1,26.0
1,-0.918024,0.047361,72.819,50.665,21.628,18.4,59.6,56.8,37338.4,31.8
2,-0.857675,-0.143112,64.542,45.385,20.943,16.8,84.9,82.6,34217.8,32.6
3,-0.831118,-0.006355,48.021,35.039,20.919,16.5,60.0,58.8,27651.4,31.4
4,-0.687750,-0.148316,43.180,31.828,20.754,20.8,46.2,44.2,20144.4,28.5


In [281]:
MostImp = reduced_working_dataframe

In [282]:

# Example DataFrame (replace with your actual DataFrame)
df = features
dfa = pd.DataFrame()
# Define feature pairs and their names
feature_pairs = [
    ("Basel Temperature [2 m elevation corrected]", "Basel Temperature [2 m elevation corrected]"),
    ("Basel Temperature [2 m elevation corrected]", "Basel Wind Speed [10 m]"),
    ("Basel Temperature [2 m elevation corrected]", "660n_sumPW"),
    ("Basel Temperature [2 m elevation corrected]", "Gasverbrauch"),
    ("Basel Temperature [2 m elevation corrected]", "Traffic"),
    ("Basel Precipitation Total", "Hour"),
    ("Basel Wind Speed [10 m]", "Basel Wind Speed [10 m]"),
    ("Basel Wind Speed [10 m]", "Basel Wind Direction [10 m]"),
    ("Basel Wind Speed [10 m]", "Gasverbrauch"),
    ("Basel Wind Speed [10 m]", "Hour"),
    ("Basel Wind Direction [10 m]", "Basel Wind Direction [10 m]"),
    ("Basel Wind Direction [10 m]", "Hour"),
    ("406v_sumLW", "408n_sumLief"),
    ("660n_sumPW", "Gasverbrauch"),
    ("Gasverbrauch", "Gasverbrauch"),
    ("Gasverbrauch", "Hour"),
]

# Loop through the pairs, calculate the product, and add to the DataFrame
for feature1, feature2 in feature_pairs:
    new_feature_name = f"{feature1}_x_{feature2}"
    dfa[new_feature_name] = df[feature1] * df[feature2]

# Save or view the updated DataFrame
# df.to_csv('updated_data.csv', index=False)
print(dfa.head())

print(f'Size of dfa: {dfa.shape}')


   Basel Temperature [2 m elevation corrected]_x_Basel Temperature [2 m elevation corrected]  \
0                                                                                   0.858340   
1                                                                                   1.085695   
2                                                                                   1.351255   
3                                                                                   1.288480   
4                                                                                   1.354144   

   Basel Temperature [2 m elevation corrected]_x_Basel Wind Speed [10 m]  \
0                                                               0.643029   
1                                                               0.956551   
2                                                               0.996992   
3                                                               0.943412   
4                                          

In [283]:

# Concatenate dfa to MostImp
MostImp_combined = pd.concat([MostImp, dfa], axis=1)

# Standardize the features
scaler = StandardScaler()
MostImp_combined_scaled = pd.DataFrame(scaler.fit_transform(MostImp_combined.iloc[:, 1:]), columns=MostImp_combined.columns[1:])
MostImp_combined_scaled.insert(0, 'Datum', MostImp_combined['Datum'])

# Export the combined DataFrame to a CSV file
MostImp_combined_scaled.to_csv('../04_WorkingDatasets/Top50Combined/50MostImp+CombinedFeatures.csv', index=False)

# Creating Lagged Values from the Hours we know

In [333]:
import pandas as pd
features = pd.read_csv('../04_WorkingDatasets/Top50Combined/50MostImp+CombinedFeatures.csv')
AllFeatures = pd.read_csv('../04_WorkingDatasets/NormalData/Working_DataFrame.csv')
target = pd.read_csv('../04_WorkingDatasets/NormalData/Target_Additive.csv')

In [334]:
# Function to replace outliers using IQR with moving average
def replace_outliers_with_moving_avg(df, column, window=3):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Calculate the moving average
    df['moving_avg'] = df[column].rolling(window=window, center=True).mean()
    
    # Replace outliers with the moving average
    df[column] = df.apply(lambda row: row['moving_avg'] if row[column] < lower_bound or row[column] > upper_bound else row[column], axis=1)
    
    # Drop the temporary moving average column
    df.drop(columns=['moving_avg'], inplace=True)
    
    return df

# Replace outliers in the target dataset
target = replace_outliers_with_moving_avg(target, 'PM10_Combined_Trend_Residual')

# Ensure the features and target datasets are aligned
features = features[features['Datum'].isin(target['Datum'])]
target = target[target['Datum'].isin(features['Datum'])]
AllFeatures = AllFeatures[AllFeatures['Datum'].isin(target['Datum'])]

print(target.head())


                       Datum  PM10_Combined_Trend_Residual
0  2022-01-01 00:00:00+00:00                           NaN
1  2022-01-01 01:00:00+00:00                     53.126838
2  2022-01-01 02:00:00+00:00                     36.328107
3  2022-01-01 03:00:00+00:00                     24.801767
4  2022-01-01 04:00:00+00:00                      9.683660


In [335]:
print(features.columns)
print(AllFeatures.columns)

Index(['Datum', 'pm10_stundenmittelwerte_ug_m3',
       'pm2_5_stundenmittelwerte_ug_m3', 'PM10 [ug/m3]', 'PM2.5 [ug/m3]',
       'NO2 [ug/m3]', 'NOX [ug/m3 eq. NO2]', 'no2_stundenmittelwerte_ug_m3',
       'CPC [1/cm3]', 'Basel Wind Direction [10 m]', 'Basel Wind Speed [10 m]',
       'Gasverbrauch', 'Basel Precipitation Total', 'SO2 [ug/m3]',
       'O3 [ug/m3]', 'TEMP [C]', 'Basel Temperature [2 m elevation corrected]',
       '408n_sumPW', 'PREC [mm]', '84111104n_sumLief', '408n_sumLief',
       '84111104v_sumLief', 'Stromverbrauch', '417n_sumPW', '408n_sumLW',
       '402v_sumPW', '403v_sumLW', '405v_sumLW', '402n_sumLW', '403v_sumPW',
       '406v_sumLW', '660n_sumPW', '660v_sumPW', '350v_sumLW', '417n_sumLW',
       '352v_sumPW', '415v_sumLW', '417n_sumLief', '419n_sumLief',
       '352v_sumLief', '420n_sumLW', '402n_sumLief', 'Traffic', '659n_sumLW',
       '406v_sumLief', '653n_sumLW', '659v_sumLW', '419n_sumPW', '406n_sumLW',
       '405v_sumLief', '419v_sumLW',
       'Basel

In [336]:
AllFeatures.head()

AllFeaturescopy = AllFeatures.copy()

In [338]:
from sklearn.preprocessing import StandardScaler
from traitlets import All

# Create lagged target variables
features['PM10_1h_lag'] = target['PM10_Combined_Trend_Residual'].shift(1)
features['PM10_2h_lag'] = target['PM10_Combined_Trend_Residual'].shift(2)
features['PM10_24h_lag'] = target['PM10_Combined_Trend_Residual'].shift(24)
AllFeatures['PM10_1h_lag'] = target['PM10_Combined_Trend_Residual'].shift(1)
AllFeatures['PM10_2h_lag'] = target['PM10_Combined_Trend_Residual'].shift(2)
AllFeatures['PM10_24h_lag'] = target['PM10_Combined_Trend_Residual'].shift(24)

# Drop rows with NaN values
features.dropna(inplace=True)
AllFeatures.dropna(inplace=True)
target = target[target['Datum'].isin(features['Datum'])]
target = target[target['Datum'].isin(AllFeatures['Datum'])]

features.drop(columns=['Datum'], inplace=True)
AllFeatures.drop(columns=['Datum'], inplace=True)

# Standardize the features dataset
scaler = StandardScaler()
features_scaled = pd.DataFrame(scaler.fit_transform(features), columns=features.columns)
AllFeatures_scaled = pd.DataFrame(scaler.fit_transform(AllFeatures), columns=AllFeatures.columns)

# Ensure there are no NaN values in the target dataset
target = target.dropna()

features_scaled.insert(0, 'Datum', target['Datum'])
AllFeatures_scaled.insert(0, 'Datum', AllFeaturescopy['Datum'])



print(features_scaled.columns)
print(AllFeatures_scaled.columns)

KeyError: 'Datum'

In [339]:
AllFeatures_scaled.head()

,Datum,Rebgassechange,Clarahuuschange,Citychange,Storchenchange,Post Baselchange,Aeschenchange,Bahnhof Südchange,Bad. Bahnhofchange,Europechange,Claramattechange,Elisabethenchange,Steinenchange,Kunstmuseumchange,Messechange,Anfoschange,Centralbahnparkingchange,Basel Temperature [2 m elevation corrected],Basel Precipitation Total,Basel Wind Speed [10 m],Basel Wind Direction [10 m],350n_sumPW,350n_sumLief,350n_sumLW,350v_sumPW,350v_sumLief,350v_sumLW,352n_sumPW,352n_sumLief,352n_sumLW,352v_sumPW,352v_sumLief,352v_sumLW,354n_sumPW,354n_sumLief,354n_sumLW,354v_sumPW,354v_sumLief,354v_sumLW,402v_sumPW,402v_sumLief,402v_sumLW,402n_sumPW,402n_sumLief,402n_sumLW,403n_sumPW,403n_sumLief,403n_sumLW,403v_sumPW,403v_sumLief,403v_sumLW,405n_sumPW,405n_sumLief,405n_sumLW,405v_sumPW,405v_sumLief,405v_sumLW,406n_sumPW,406n_sumLief,406n_sumLW,406v_sumPW,406v_sumLief,406v_sumLW,407n_sumPW,407n_sumLief,407n_sumLW,408n_sumPW,408n_sumLief,408n_sumLW,409n_sumPW,409n_sumLief,409n_sumLW,409v_sumPW,409v_sumLief,409v_sumLW,410v_sumPW,410v_sumLief,410v_sumLW,410n_sumPW,410n_sumLief,410n_sumLW,411n_sumPW,411n_sumLief,411n_sumLW,411v_sumPW,411v_sumLief,411v_sumLW,412v_sumPW,412v_sumLief,412v_sumLW,413n_sumPW,413n_sumLief,413n_sumLW,415v_sumPW,415v_sumLief,415v_sumLW,415n_sumPW,415n_sumLief,415n_sumLW,417n_sumPW,417n_sumLief,417n_sumLW,419v_sumPW,419v_sumLief,419v_sumLW,419n_sumPW,419n_sumLief,419n_sumLW,420n_sumPW,420n_sumLief,420n_sumLW,420v_sumPW,420v_sumLief,420v_sumLW,653v_sumPW,653v_sumLief,653v_sumLW,653n_sumPW,653n_sumLief,653n_sumLW,656v_sumPW,656v_sumLief,656v_sumLW,659v_sumPW,659v_sumLief,659v_sumLW,659n_sumPW,659n_sumLief,659n_sumLW,660v_sumPW,660v_sumLief,660v_sumLW,660n_sumPW,660n_sumLief,660n_sumLW,84111104n_sumPW,84111104n_sumLief,84111104n_sumLW,84111104v_sumPW,84111104v_sumLief,84111104v_sumLW,84111108n_sumPW,84111108n_sumLief,84111108n_sumLW,84111108v_sumPW,84111108v_sumLief,84111108v_sumLW,Gasverbrauch,Hour,Traffic,Stromverbrauch,o3_stundenmittelwerte_ug_m3,pm10_stundenmittelwerte_ug_m3,pm2_5_stundenmittelwerte_ug_m3,no2_stundenmittelwerte_ug_m3,O3 [ug/m3],NO2 [ug/m3],PM10 [ug/m3],PM2.5 [ug/m3],CPC [1/cm3],PREC [mm],RAD [W/m2],SO2 [ug/m3],NOX [ug/m3 eq. NO2],TEMP [C],PM10_1h_lag,PM10_2h_lag,PM10_24h_lag
0,2022-01-01 00:00:00+00:00,-0.069723,-0.000039,-0.010908,-0.175172,0.000008,0.000005,-0.245170,-0.052815,-0.000129,9.531997e-17,-0.355258,-0.232618,-0.000061,-0.275864,0.000016,0.000030,-1.245999,-0.319143,-0.274121,-0.072930,-1.420082,-1.029477,-0.796137,-1.387612,-1.133338,-0.888010,-1.267946,-0.918964,-1.114793,-1.357989,-1.057661,-1.125040,-1.167343,-0.973562,-0.780311,-1.260781,-0.888802,-0.513252,-1.378962,-0.964847,-1.044521,-1.372207,-1.072556,-1.104278,-1.175188,-0.930064,-1.170472,-1.140982,-0.891770,-1.123793,-1.409680,-0.927787,-0.767146,-1.347303,-0.788647,-0.787696,-1.474792,-0.997484,-1.079606,-1.430854,-1.041960,-0.978201,-1.397263,-0.917100,-0.713090,-1.162183,-0.924398,-1.390794,-1.314711,-1.063034,-1.356508,-1.322971,-1.015252,-1.256865,-1.262183,-0.895449,-0.963542,-1.222256,-0.723496,-0.844695,-1.251113,-0.843933,-1.267186,-1.305995,-0.871815,-1.236483,-1.297555,-0.880995,-1.216307,-1.304800,-0.889927,-1.231113,-1.352788,-1.015775,-0.998295,-1.518409,-1.174801,-0.929518,-1.027889,-0.794816,-1.066891,-1.358693,-0.819220,-0.84902,-1.193806,-1.065470,-0.806285,-1.438825,-0.976268,-0.982533,-1.433507,-1.010154,-1.004656,-1.376217,-0.902597,-1.315519,-1.145131,-0.920061,-1.440249,-1.401732,-0.902492,-0.692094,-1.210137,-0.944220,-0.86775,-1.255675,-1.009888,-0.819096,-1.587458,-1.514598,-1.301145,-1.709565,-1.458986,-1.289054,-1.260178,-0.156661,-0.676879,-1.364838,-0.137416,-0.746088,-1.098649,-0.776114,-1.056601,-1.201719,-0.649637,-1.07371,0.478518,-1.516949,-1.294425,-1.601434,-0.614359,0.371266,0.764582,-0.244192,-1.754534,1.257089,0.584737,1.069597,0.510222,-0.152269,-0.639512,0.295487,1.099745,-1.039634,0.096166,0.010001,5.637085
1,2022-01-01 01:00:00+00:00,0.000222,-0.000039,-0.000018,0.000017,0.000008,0.000005,0

In [340]:
target_mean = target['PM10_Combined_Trend_Residual'].mean()
print(f"The mean of the target variable is: {target_mean}")

The mean of the target variable is: 14.4243556847222


In [288]:
features_scaled.to_csv('../04_WorkingDatasets/Top50CombLagged/50CombLagged.csv', index=False)
AllFeatures_scaled.to_csv('../04_WorkingDatasets/NormalDataLaggedTarget/WDFTargetLagged.csv', index=False)
target.to_csv('../04_WorkingDatasets/Top50CombLagged/TargetOutliersTreated.csv', index=False)
target.to_csv('../04_WorkingDatasets/NormalDataLaggedTarget/TargetOutliersTreated.csv', index=False)

# Add the combined Features to the Normal DataFrames

In [341]:
NormalData_combined = pd.concat([final_df, dfa], axis=1)

scaler = StandardScaler()
MostImp_combined_scaled = pd.DataFrame(scaler.fit_transform(NormalData_combined.iloc[:, 1:]), columns=NormalData_combined.columns[1:])
MostImp_combined_scaled.insert(0, 'Datum', NormalData_combined['Datum'])

# Export the combined DataFrame to a CSV file
MostImp_combined_scaled.to_csv('../04_WorkingDatasets/01 NormalComb/NormalCombined.csv', index=False)
target.to_csv("../04_WorkingDatasets/01 NormalComb/Target_Additive.csv", index=False)


In [342]:
NormalData_combined_Lagged = pd.concat([AllFeatures_scaled, dfa], axis=1)

scaler = StandardScaler()
MostImp_combined_scaled = pd.DataFrame(scaler.fit_transform(NormalData_combined_Lagged.iloc[:, 1:]), columns=NormalData_combined_Lagged.columns[1:])
MostImp_combined_scaled.insert(0, 'Datum', NormalData_combined_Lagged['Datum'])

# Export the combined DataFrame to a CSV file
MostImp_combined_scaled.to_csv('../04_WorkingDatasets/02 NormalCombLagged/WDFCombLagged.csv', index=False)
target.to_csv('../04_WorkingDatasets/02 NormalCombLagged/TargetOutliersTreated.csv', index=False)
